In [61]:
import json
import requests
import pprint

1. create context (dialogue history)

In [72]:
context = \
"""We are at the leaning tower of Pisa. 
Who built it? 
Giuseppe Napolitano built this tower. 
When was it built? 
In the nineteenth century. 
Who was he married to?
He was married to Isabella d'Aglio.
"""

2. create entities

In [73]:
poi_start = context.find("the leaning tower of Pisa")
poi_end = poi_start + len("the leaning tower of Pisa")
builder_start = context.find("Giuseppe Napolitano")
builder_end = builder_start + len("Giuseppe Napolitano")

entities = {
    "$POI": {"span": [poi_start, poi_end], "type": "building"},
    "$BUILDER": {"span": [builder_start, builder_end]}
}

3. create question

In [74]:
sentence = "When was it built?"

4. create request and execute

In [75]:
data = {"context": context,
        "entities": entities,
        "sentence": sentence}
data_json = json.dumps(data)

payload = {"data": data_json}
r = requests.get("http://localhost:6007/entitygetcoref", params=payload)


In [76]:
pp = pprint.PrettyPrinter(indent=3)
pp.pprint(json.loads(r.text))

{  'context': 'We are at the leaning tower of Pisa. \n'
              'Who built it? \n'
              'Giuseppe Napolitano built this tower. \n'
              'When was it built? \n'
              'In the nineteenth century. \n'
              'Who was he married to?\n'
              "He was married to Isabella d'Aglio.\n",
   'references': [  {  'from': {'end_char': 11, 'start_char': 9, 'text': 'it'},
                       'to': '$POI'}],
   'resolved_sentence': 'When was the leaning tower of Pisa built?',
   'sentence': 'When was it built?'}


### Return format:
* "context": is original context string
* "sentence": is original sentence string
* "resolved_sentence": is sentence where referring expression is replaced by its target. If resolved by neural-coref, replaces referring expression with its target. If resolved by manual POI rules, replaces referring expression with "$POI"
* "references": list of dictionaries with source "from" and target "to"
    *  "from": from which span in "sentence" the reference goes
        * "text": string of referring expression
        * "start_char": start of span in "sentence", character index
        * "end_char": end of span in "sentence", character index
    * "to": to which span in "context" or to which entity from "entities" GET request data the reference goes. Return can be a string, in which case an entity name from the "entities" GET request argument is used. Otherwise, follows span specification:
        * "text": string of target expression
        * "start_char": start of span in "context", character index
        * "end_char": end of span in "context", character index

(in a function:)

In [77]:
def do_question(sentence):
    data = {"context": context,
            "entities": entities,
            "sentence": sentence}
    data_json = json.dumps(data)

    payload = {"data": data_json}
    r = requests.get("http://localhost:6007/entitygetcoref", params=payload)
    return json.loads(r.text)

In [78]:
pp.pprint(do_question("When was he born?"))

{  'context': 'We are at the leaning tower of Pisa. \n'
              'Who built it? \n'
              'Giuseppe Napolitano built this tower. \n'
              'When was it built? \n'
              'In the nineteenth century. \n'
              'Who was he married to?\n'
              "He was married to Isabella d'Aglio.\n",
   'references': [  {  'from': {'end_char': 11, 'start_char': 9, 'text': 'he'},
                       'to': '$BUILDER'}],
   'resolved_sentence': 'When was Giuseppe Napolitano born?',
   'sentence': 'When was he born?'}


In [79]:
pp.pprint(do_question("Who was his wife?"))

{  'context': 'We are at the leaning tower of Pisa. \n'
              'Who built it? \n'
              'Giuseppe Napolitano built this tower. \n'
              'When was it built? \n'
              'In the nineteenth century. \n'
              'Who was he married to?\n'
              "He was married to Isabella d'Aglio.\n",
   'references': [  {  'from': {'end_char': 11, 'start_char': 8, 'text': 'his'},
                       'to': '$BUILDER'}],
   'resolved_sentence': 'Who was Giuseppe Napolitano wife?',
   'sentence': 'Who was his wife?'}


In [80]:
pp.pprint(do_question("What did she do"))

{  'context': 'We are at the leaning tower of Pisa. \n'
              'Who built it? \n'
              'Giuseppe Napolitano built this tower. \n'
              'When was it built? \n'
              'In the nineteenth century. \n'
              'Who was he married to?\n'
              "He was married to Isabella d'Aglio.\n",
   'references': [  {  'from': {'end_char': 12, 'start_char': 9, 'text': 'she'},
                       'to': '$BUILDER'}],
   'resolved_sentence': 'What did Giuseppe Napolitano do',
   'sentence': 'What did she do'}


In [85]:
pp.pprint(do_question("Who designed this?"))

{  'context': 'We are at the leaning tower of Pisa. \n'
              'Who built it? \n'
              'Giuseppe Napolitano built this tower. \n'
              'When was it built? \n'
              'In the nineteenth century. \n'
              'Who was he married to?\n'
              "He was married to Isabella d'Aglio.\n",
   'references': [  {  'from': {  'end_char': 17,
                                  'start_char': 13,
                                  'text': 'this'},
                       'to': '$POI'}],
   'resolved_sentence': 'Who designed $POI?',
   'sentence': 'Who designed this?'}


In [86]:
pp.pprint(do_question("Who designed this tower?"))

{  'context': 'We are at the leaning tower of Pisa. \n'
              'Who built it? \n'
              'Giuseppe Napolitano built this tower. \n'
              'When was it built? \n'
              'In the nineteenth century. \n'
              'Who was he married to?\n'
              "He was married to Isabella d'Aglio.\n",
   'references': [  {  'from': {  'end_char': 23,
                                  'start_char': 13,
                                  'text': 'this tower'},
                       'to': '$POI'}],
   'resolved_sentence': 'Who designed the leaning tower of Pisa?',
   'sentence': 'Who designed this tower?'}
